In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/ML_Project/tweets.csv"
df = pd.read_csv(path)

In [ ]:
df = df.dropna(subset=['text'])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
total_words = len(tokenizer.word_index) + 1

In [ ]:
sequences = tokenizer.texts_to_sequences(df['text'])
padded_sequences = pad_sequences(sequences)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['target'], test_size=0.2, random_state=42)

In [ ]:
embedding_dim = 50
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=embedding_dim, input_length=padded_sequences.shape[1]))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
285/285 [==============================] - 23s 69ms/step - loss: 0.3834 - accuracy: 0.8538 - val_loss: 0.2753 - val_accuracy: 0.9006
Epoch 2/5
285/285 [==============================] - 19s 65ms/step - loss: 0.1742 - accuracy: 0.9379 - val_loss: 0.2638 - val_accuracy: 0.8993
Epoch 3/5
285/285 [==============================] - 19s 66ms/step - loss: 0.0704 - accuracy: 0.9779 - val_loss: 0.3892 - val_accuracy: 0.8945
Epoch 4/5
285/285 [==============================] - 19s 67ms/step - loss: 0.0328 - accuracy: 0.9901 - val_loss: 0.3787 - val_accuracy: 0.8896
Epoch 5/5
285/285 [==============================] - 19s 66ms/step - loss: 0.0133 - accuracy: 0.9963 - val_loss: 0.4366 - val_accuracy: 0.8909


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

72/72 [==============================] - 1s 14ms/step - loss: 0.4366 - accuracy: 0.8909
Accuracy: 89.09%


In [ ]:
test_sentence = ["No cows today but our local factory is sadly still ablaze"]

test_sequences = tokenizer.texts_to_sequences(test_sentence)
padded_test_sequences = pad_sequences(test_sequences, maxlen=padded_sequences.shape[1])

predictions = model.predict(padded_test_sequences)

threshold = 0.5
if predictions[0, 0] >= threshold:
    print(f'Test sentence: "{test_sentence[0]}" is a disaster.')
else:
    print(f'Test sentence: "{test_sentence[0]}" is not a disaster.')

1/1 [==============================] - 0s 46ms/step
Test sentence: "No cows today but our local factory is sadly still ablaze" is a disaster.


In [ ]:
test_sentence = "There is a cyclone in Florida"
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(test_sentence)
locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
print("Disaster Locations:", locations)

Disaster Locations: ['Florida']


In [ ]:
import pickle
with open('/content/drive/MyDrive/ML_Project/lstm.pkl', 'wb') as f:
    pickle.dump(model, f)